In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df = pd.read_csv('1980.csv')

In [ ]:
df['genres'].unique()

Check for duplicate values

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.shape

There were a lot of duplicate values..

Check for null values

In [ ]:
df.isna().any()

There exist Null values for the Chart Power and for the isrc. The null values for the Chart Power are valid, because not every song was in the Charts. Lets have a look at the null values for isrc:

In [ ]:
df[df['isrc'].isna()]

In [ ]:
df_categorical = df[['spotify_id', 'genres', 'name', 'artists', 'album', 'release_date', 'release_date_precision', 'uri', 'isrc']]

In [ ]:
display(df_categorical.head())
df_categorical.shape

In [ ]:
df_categorical = df_categorical.groupby('spotify_id').agg(lambda x: list(x)).reset_index()

In [ ]:
display(df_categorical.head())
df_categorical.shape

In [ ]:
df_numerical = df.drop(columns=['genres', 'name', 'artists', 'album', 'release_date', 'release_date_precision', 'uri', 'isrc'])

In [ ]:
df_numerical.drop_duplicates(inplace=True)

In [ ]:
df_numerical.head()

In [ ]:
df_numerical.shape

In [ ]:
df_numerical = df_numerical.groupby('spotify_id').mean()

In [ ]:
df_numerical.shape

In [ ]:
df_numerical.reset_index(inplace=True)

In [ ]:
df_categorical = df_categorical.merge(df_numerical, how='left', on='spotify_id')

In [ ]:
df_categorical

Create a DataFrame which contains only the relevant features for the recommender system

In [ ]:
df_recommender = df.drop(columns=['artists', 'genres','album', 'release_date', 'release_date_precision', 'chart_power', 'uri', 'popularity', 'name', 'isrc'])

Drop duplicate values..

In [ ]:
df_recommender.shape

In [ ]:
df_recommender.drop_duplicates(inplace=True)

In [ ]:
df_recommender.shape

There were a lot of duplicates..

In [ ]:
df_recommender.head()

Since all features except the spotify_id are numerical features and the spotify_id should be an unique identifier, try to group by the id to reduce the DataFrame even more.

In [ ]:
df_recommender = df_recommender.groupby(['spotify_id']).mean()

In [ ]:
display(df_recommender.head())
df_recommender.shape

The shape stayed the same, so there were no duplicate ids.

Scale the data, so every feature has the same influence.

In [ ]:
scaler = MinMaxScaler()
df_recommender_scaled = scaler.fit_transform(df_recommender)
df_recommender_scaled = pd.DataFrame(df_recommender_scaled, columns=df_recommender.columns, index = df_recommender.index)

In [ ]:
df_recommender_scaled.head()

In [ ]:
df_recommender_scaled.reset_index(inplace=True)
df_recommender.reset_index(inplace=True)

Convert every genre to a feature. If a song is part of a genre it should contain the value 1 otherwise 0.

In [ ]:
ct = pd.crosstab(df['spotify_id'], df['genres'])
ct.reset_index(inplace=True)

In [ ]:
display(ct.head())
ct.shape

Merge both DataFrames together to create the Recommender System.

In [ ]:
ct_merged = ct.merge(df_recommender_scaled, on=['spotify_id'], how='left')

In [ ]:
ct_merged.set_index(['spotify_id'], inplace=True)

In [ ]:
display(ct_merged.head())
ct_merged.shape

Overall there are 108 features used for the Recommendation system.

Compute the cosine similarity between each song. A value close to one means that 2 two songs are equal a value close to 0 means, that they are completley different.

In [ ]:
cosine_similarity_array = cosine_similarity(ct_merged)

In [ ]:
cosine_similarity_array.shape

In [ ]:
cosine_similarity_df = pd.DataFrame(cosine_similarity_array, index=ct_merged.index, columns=ct_merged.index)

In [ ]:
cosine_similarity_df

In [ ]:
df.sort_values('popularity', ascending=False).head(10)

In [ ]:
def recommend_tracks(track: str):
    '''
    Recommends tracks that are similar to the provided track.

    Parameter
    ---------
    track: str
        Provided track
    
    '''
    ids = df[df.name == track]['spotify_id'].values
    for id in ids:
        recommender_df = cosine_similarity_df[[id]]
        recommender_df.reset_index(inplace=True)
        recommender_df = recommender_df.rename(columns={id: 'Score'})
        recommender_df = recommender_df.merge(df_categorical, how='left', on='spotify_id')
        display(recommender_df.sort_values(by='Score', ascending=False).head(10))


In [ ]:
recommend_tracks('Back In Black')